In [1]:
import gc
from all_stand_var import conv_dict, vent_cols3
from all_own_funct import memory_downscale,memory_upscale
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from seaborn import heatmap
from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
from scipy.stats import kurtosis
from matplotlib.backends.backend_pdf import PdfPages
import keras.backend as K
from RNN_LTSM_CHD import return_loaded_model
import pickle
import locale

from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix

locale.setlocale(locale.LC_ALL, 'fr_FR')
output_folder = os.path.join(os.getcwd(), 'Results_CHD_v5','final')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

vent_cols3 = ['pat_hosp_id', 'pat_bd', 'pat_datetime', 'OK_datum', 'mon_rr', 'mon_hr', 'mon_sat',
              'mon_etco2', 'vent_m_fio2', 'vent_m_ppeak','vent_m_peep',
             'mon_ibp_mean','pat_weight_act','vent_m_rr', 'vent_m_tv_exp']
dtype_dict={'vent_cat': 'category','vent_machine':'category','vent_mode':'category'}
df_raw=pd.read_csv(r'data\CHD_between.csv',delimiter=';',decimal=',',converters=conv_dict,usecols=vent_cols3,dtype=dtype_dict,dayfirst=True,parse_dates=['pat_bd','pat_datetime','OK_datum'],na_values=['NULL','null', 'Null','nUll','nuLl','nulL'],chunksize=15000000)


import LR_build_CHD as pp
df_list=[]
a=[]

for chunk in df_raw:
    df = pp.data_pp_function(chunk)
    df_list.append(df)
    a.append(chunk.groupby(['pat_hosp_id','OK_datum'],sort=False).ngroup().max())
    del df
print(np.sum(a))
#df=df.groupby('Admissionnumber',sort=False).fillna(method='ffill').fillna(method='bfill')
df=pd.concat(df_list,sort=False,ignore_index=False)
df['Adnum'] = df.groupby(['pat_hosp_id', 'OK_datum'], sort=False, as_index=False).ngroup()
print(len(df['Adnum'].unique()))
import tables
df.sort_values(['pat_datetime'], inplace=True)
df=memory_downscale(df)
#df = pp.time_step_df(df=df, timestep='12H')
#print(df['idx'].value_counts())
df.to_hdf(os.path.join(output_folder, 'processed_df.h5'),key='df',mode='w',format='table', complib='zlib', complevel=9)

C:\Users\berend\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)
c:\Users\berend\Documents\Python_Scripts\Patient_Selection\LR_build_CHD.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pat_datetime_temp'] = pd.to_datetime(df['pat_datetime']).dt.date
c:\Users\berend\Documents\Python_Scripts\Patient_Se

In [12]:
import tables
b=df['Adnum'].value_counts()
c=b.loc[b.values < 10]
print(len(c))
print(c)

9
1468    9
400     7
1719    6
584     6
528     6
2024    5
529     3
1905    3
1587    2
Name: Adnum, dtype: int64


In [13]:
from all_own_funct import memory_upscale
print(df.info())
df=df.reset_index(drop=False)
df=memory_upscale(df)
print(len(df['Adnum'].unique()))
def fill_group(group):
    group=group.fillna(method='ffill',axis=1).fillna(method='bfill')
    return group
df=df.groupby(['pat_hosp_id','OK_datum','Adnum'],sort=False,as_index=False).apply(fill_group)
print(df.info())
print(len(df['Adnum'].unique()))

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 978473 entries, (0, 7171037, Timestamp('2008-04-11 00:00:00')) to (77, 3536787, Timestamp('2020-01-10 00:00:00'))
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   pat_datetime     978473 non-null  datetime64[ns]
 1   pat_weight_act   978473 non-null  float16       
 2   mon_etco2        829237 non-null  float16       
 3   mon_hr           971134 non-null  float16       
 4   mon_ibp_mean     887213 non-null  float16       
 5   mon_rr           931288 non-null  float16       
 6   mon_sat          959128 non-null  float16       
 7   vent_m_fio2      973620 non-null  float16       
 8   vent_m_peep      960680 non-null  float16       
 9   vent_m_ppeak     977208 non-null  float16       
 10  vent_m_rr        978466 non-null  float16       
 11  vent_m_tv_exp    972433 non-null  float16       
 12  Age              978473 non-null  float16 

In [14]:
print(df['Adnum'].value_counts())
b=df['Adnum'].value_counts()
c=b.loc[b.values < 700]
print(len(c))
print(c)

2070    1440
755      720
1373     720
1184     720
672      720
        ... 
528        6
2024       5
529        3
1905       3
1587       2
Name: Adnum, Length: 2149, dtype: int64
1084
1857    696
195     682
1300    679
264     675
399     661
       ... 
528       6
2024      5
529       3
1905      3
1587      2
Name: Adnum, Length: 1084, dtype: int64
